In [1]:
!pip install python-decouple
!pip install openai==1.3.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
os.environ["OPENAI_API_KEY"] = "***********************************************"
os.environ["HUGGINGFACE_API_TOKEN"] = "*******************************************"

In [3]:
import sys, json
import pandas as pd
from pathlib import Path
from IPython.display import HTML, display
from decouple import config
sys.path.append("../src/")
from llm_helpers import chatgpt_ask

pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)

#data_dir = Path(".").absolute().parent/"data"
data_dir = Path("data")
print(data_dir)
ls = lambda p:print("\n".join(map(str,p.iterdir())))

ls(data_dir)

data
data/sample_apps.parquet


In [4]:
df = pd.read_parquet(data_dir / "sample_apps.parquet").sample(9)
df

,bundle_id,title,description,store_url,category_names,ios
39810,com.playrix.gardenscapes,Gardenscapes,Welcome to Gardenscapes—the first hit from Pla...,https://play.google.com/store/apps/details?id=...,"GAME_CASUAL,GAME",False
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False
36869,com.nexters.herowars,Hero Wars – Fantasy Battles,Unlock skills and battle enemies with Hero War...,https://play.google.com/store/apps/details?id=...,"GAME_ROLE_PLAYING,GAME",False
59980,net.peakgames.toonblast,Toon Blast,From the creators of Toy Blast comes the ultim...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
7477,com.alibaba.aliexpresshd,AliExpress,Maximum deals. Maximum fun. Shop our biggest s...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False
34033,com.mistplay.mistplay,MISTPLAY: Play to earn rewards,"Introducing Mistplay, the app that lets you ea...",https://play.google.com/store/apps/details?id=...,"ENTERTAINMENT,APPLICATION",False
35146,com.moonactive.coinmaster,Coin Master,Join your Facebook friends and millions of pla...,https://play.google.com/store/apps/details?id=...,"GAME_CASUAL,GAME",False
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True
15820,com.creditkarma.mobile,Credit Karma,• Check your free credit scores – Learn what a...,https://play.google.com/store/apps/details?id=...,"FINANCE,APPLICATION",False


## Ask ChatGPT a question on every row

In [5]:
df["for_kids"] = df.apply(chatgpt_ask("Given the app description:\n{description}\n\nIs it for kids?\nAnswer only yes or no"),axis=1)
df

,bundle_id,title,description,store_url,category_names,ios,for_kids
39810,com.playrix.gardenscapes,Gardenscapes,Welcome to Gardenscapes—the first hit from Pla...,https://play.google.com/store/apps/details?id=...,"GAME_CASUAL,GAME",False,No.
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,No
36869,com.nexters.herowars,Hero Wars – Fantasy Battles,Unlock skills and battle enemies with Hero War...,https://play.google.com/store/apps/details?id=...,"GAME_ROLE_PLAYING,GAME",False,Yes
59980,net.peakgames.toonblast,Toon Blast,From the creators of Toy Blast comes the ultim...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,Yes.
7477,com.alibaba.aliexpresshd,AliExpress,Maximum deals. Maximum fun. Shop our biggest s...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,No
34033,com.mistplay.mistplay,MISTPLAY: Play to earn rewards,"Introducing Mistplay, the app that lets you ea...",https://play.google.com/store/apps/details?id=...,"ENTERTAINMENT,APPLICATION",False,No
35146,com.moonactive.coinmaster,Coin Master,Join your Facebook friends and millions of pla...,https://play.google.com/store/apps/details?id=...,"GAME_CASUAL,GAME",False,No
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True,No.
15820,com.creditkarma.mobile,Credit Karma,• Check your free credit scores – Learn what a...,https://play.google.com/store/apps/details?id=...,"FINANCE,APPLICATION",False,No


## Question:
Please rewrite the code such that the response to a yes/no question would be boolean (`True` or `False`)

In [6]:
def true_false(x):
    if "yes" in x.lower():
        return True
    else:
        return False

df["for_kids"] = df["for_kids"].apply(true_false)

In [8]:
df

,bundle_id,title,description,store_url,category_names,ios,for_kids
39810,com.playrix.gardenscapes,Gardenscapes,Welcome to Gardenscapes—the first hit from Pla...,https://play.google.com/store/apps/details?id=...,"GAME_CASUAL,GAME",False,False
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,False
36869,com.nexters.herowars,Hero Wars – Fantasy Battles,Unlock skills and battle enemies with Hero War...,https://play.google.com/store/apps/details?id=...,"GAME_ROLE_PLAYING,GAME",False,True
59980,net.peakgames.toonblast,Toon Blast,From the creators of Toy Blast comes the ultim...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,True
7477,com.alibaba.aliexpresshd,AliExpress,Maximum deals. Maximum fun. Shop our biggest s...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,False
34033,com.mistplay.mistplay,MISTPLAY: Play to earn rewards,"Introducing Mistplay, the app that lets you ea...",https://play.google.com/store/apps/details?id=...,"ENTERTAINMENT,APPLICATION",False,False
35146,com.moonactive.coinmaster,Coin Master,Join your Facebook friends and millions of pla...,https://play.google.com/store/apps/details?id=...,"GAME_CASUAL,GAME",False,False
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True,False
15820,com.creditkarma.mobile,Credit Karma,• Check your free credit scores – Learn what a...,https://play.google.com/store/apps/details?id=...,"FINANCE,APPLICATION",False,False
